<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Create scalable in-database Semantic Search capabilities powered by open-source models and native ClearScape Analytics functions
  <br>
       <img id="teradata-logo" src="images/TeradataLogo.png" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<hr>

<b style = 'font-size:28px;font-family:Arial;color:#00233C'>Demonstrations Overview</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstrations illustrate the end-to-end process of how organizations can utilize VantageCloud Lake <b>GPU-enabled Analytic Cluster</b> architecture to run open-source large language models at massive parallelism and scale, and then leverage these next-generation capabilities in ad-hoc analytics, development, and operational processing.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This notebook contains several high-level demonstrations that can be run together or individually, and are designed to illustrate;</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Container Management</b>.  Administrators can create and manage <b>secure, custom</b> runtime containers that will host any number of models and model artifacts to unlock GPU-augmented analytics</li>
    <li><b>Data Prep with Sentiment Extraction</b>. Developers will use the Hugging Face cardiffnlp/twitter-roberta-base-sentiment-latest model to extract user sentiment from Call Center transcripts</li>
    <li><b>Operationalization</b>. Combine the data prep and transformation steps with powerful native <b>ClearScape Analytics</b> functions against the data sets to create a scalable, on-demand Sentiment extraction function</li>
    </ol>

<b style = 'font-size:28px;font-family:Arial;color:#00233C'>End-to-End workflow</b>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The high level process is as follows:</p>

<table style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'>
    <tr><td style = 'vertical-align:top' width = '40%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>The Data Scientist conducts analytics activities using his or her own python tools and packages of choice, then connects to VantageCloud Lake through teradataml client library and teradatasql python driver.</li>
                <br>      
                <li>Teradataml provides APIs to create and manage custom runtime environments; including custom libraries, dependencies, model artifacts, and scoring scripts.  The user can leverage these APIs to create one or many custom, dedicated environments to host their code.</li>
                <br>
                <li>The Data Scientist will then execute their pipeline that will;
                    <ul style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'><li>Call ClearScape Analytics functions on Compute Clusters (data prep, transformation, etc.)</li>
                        <li>Prepared data is passed to the python container running in parallel on cluster nodes.</li>
                        <li>Results (inference/predictions) are returned as "virtual" dataframes; where the data resides <b>in Vantage</b></li>
                    </ul></li>
                <br>
                <li>Worfklow can be operationalized using SQL, and results can be returned to common BI tools, persisted as part of an ETL process, or embedded in application code</li>
            </ol>
        </td><td><img src = 'images/BYOLLM_Overview.png' width = '600'></td></tr>
</table>
<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python Package Installation</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If necessary, install required client packages for the demonstrations.  User may need to restart the Jupyter kernel after installation.</p> 

In [ ]:
%pip install -r requirements.txt

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python Package Imports</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Standard practice to import required packages and libraries; execute this cell to import packages for Teradata automation as well as machine learning, analytics, utility, and data management packages.</p> 

In [ ]:
from teradataml import *
from oaf_utils import *
from teradatasqlalchemy.types import *
from time import sleep
import pandas as pd
import csv, sys, os, warnings
#from os.path import expanduser
from collections import OrderedDict

from IPython.display import clear_output , display as ipydisplay
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True

# load vars json
with open('vars.json', 'r') as f:
    session_vars = json.load(f)

# Database login information
host = session_vars['environment']['host']
username = session_vars['hierarchy']['users']['business_users'][1]['username']
password = session_vars['hierarchy']['users']['business_users'][1]['password']

# UES Authentication information
ues_url = session_vars['environment']['UES_URI']
configure.ues_url = ues_url
pat_token = session_vars['hierarchy']['users']['business_users'][1]['pat_token']
pem_file = session_vars['hierarchy']['users']['business_users'][1]['key_file']


compute_group = session_vars['hierarchy']['users']['business_users'][1]['compute_group']


# get the current python version to match deploy a custom container
python_version = str(sys.version_info[0]) + '.' + str(sys.version_info[1])
print(f'Using Python version {python_version} for user environment')


# Hugging Face model for the demo
model_name = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

# a list of required packages to install in the custom OAF container
# modify this if using different models or design patterns
pkgs = ['transformers',
        'torch',
        'sentencepiece',
        'pandas',
        'sentence-transformers',
        'dill']

# container name - set here for easier notebook navigation
### User will also be asked to change it ###
oaf_name = 'oaf_sentiment_demo'
###########################

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Connect to the database</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>After connecting, check cluster status. Start it if necessary - note the cluster only needs to be running to execute the APPLY sections of the demo.</p> 

In [ ]:
# check for existing connection
eng = check_and_connect(host=host, username=username, password=password, compute_group = compute_group)
print(eng)

# check cluster status
res = check_cluster_start(compute_group = compute_group)

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Demo 1 - Container Management
  <br>
    </p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Teradata Vantage Python Client Library provides simple, powerful methods for the creation and maintenance of custom Python runtime environments <b>in the VantageCloud environment</b> .  This allows practitioners complete control over the behavior and quality of their model performance and analytic accuracy running on the Analytic Cluster.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Custom environments are persistent.</b> Users only need to create these once and then can be saved, updated, or modified only as needed.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Container Management Process</b></p>
<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '40%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Create a unique User Environment based on available base images</li>
                <br>
                <li>Install libraries</li>
                <br>
                <li>Install models and additional user artifacts</li>
            </ol>
        </td>
        <td><img src = 'images/OAF_Env.png' width = '600'></td>
    </tr>
</table>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.  Connect to the Environment Service</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>To better support integration with Cloud Services and commong automation tools; the <b > User Environment Service</b> is accessed via RESTful APIs.  These APIs can be called directly or in the examples shown below that leverage the Python Package for Teradata (teradataml) methods.</p> 

In [ ]:
# check to see if there is a valid UES auth
# if not, authenticate
try:
    demo_env = get_env(oaf_name)
    print('Existing valid UES token')

except Exception as e:
    if '''NoneType' object has no attribute 'value''' or '''Failed to execute get_env''' in str(e):
        if set_auth_token(ues_url = ues_url, username = username, pat_token = pat_token, pem_file = pem_file):
            print('UES Authentication successful')
        else:
            print('UES Authentication failed, check URL and account info')
        pass
    else:
        raise
    

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.  Create a Custom Container in Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If desired, the user can create a <b>new</b> custom environment by starting with a "base" image and customizing it.  The steps are:</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>List the available "base" images the system supports</li>
    <li>List any existing "custom" environments the user has created</li>
    <li>If there are no custom environments, then create a new one from a base image</li>
    </ul>

In [ ]:
# Create a new environment, or connect to an existing one


try:
    ipydisplay(list_user_envs())
except Exception as e:
    if str(e).find('No user environments found') > 0:
        print('No user environments found')
        pass
    else:
        raise

print('Use an existing environment, or create a new one:')
print(f'OAF Environment is set to {oaf_name}.')
print('Enter to accept, or input a new value.')
print('If the environment is not in the list, a new one will be created')
i = input()
if len(i) != 0:
    oaf_name = i
    print(f'OAF Environment is now {oaf_name}')

try:
    demo_env = create_env(env_name = oaf_name,
                      base_env = f'python_{python_version}',
                      desc = 'OAF Demo env for LLM')
except Exception as e:
    if str(e).find('same name already exists') > 0:
        print('Environment already exists, obtaining a reference to it')
        demo_env = get_env(oaf_name)
        pass
    elif 'Invalid value for base environment name' in str(e):
        print('Unsupported base environment version, using defaults')
        demo_env = create_env(env_name = oaf_name,
                      desc = 'OAF Demo env for LLM')
    else:
        raise

# Note create_env seems to be asynchronous - sleep a bit for it to register
sleep(5)

try:
    ipydisplay(list_user_envs())
except Exception as e:
    if str(e).find('No user environments found') > 0:
        print('No user environments found')
        pass
    else:
        raise

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.  Install Dependencies</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The second step in the customization process is to install Python package dependencies. This demonstration uses the Hugging Face <a href = 'https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest'>twitter-roberta-base-sentiment</a> Sentiment Extractor.  Since VantageCloud Lake Analytic Clusters are secured by default against unauthorized access to the outside network, the user can load the required libraries and model using teradataml methods:
</p> 

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>List the currently installed models and python libraries</li>
    <li><b>If necessary</b>, install any required packages</li>
    <li><b>If necessary</b>, install the pre-trained model.  This process takes several steps;
        <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
            <li>Import and download the model</li>
            <li>Create a zip archive of the model artifacts</li>
            <li>Call the install_model() method to load the model to the container</li>
        </ol></li>
    </ul>

In [ ]:
ipydisplay(demo_env.models)

# just showing a sample here - remove .head(5) to see them all
ipydisplay(demo_env.libs.head(5))

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>A note on package versions</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Some demonstrations make use of the DataFrame apply() method, which automatically passes the python code to the Analytic Cluster.  As such, one needs to ensue the python package versions match.  dill and pandas are required, as is any additional libraries for the use case.
</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note</b> while not required for many OAF use cases, for this demo the required packages for the model execution should be installed in the local environment first.</p>

In [ ]:
# Install any Python add-ons needed by the script in the user environment
# Using option asynchronous=True for an asychronous execution of the statement.
# Note: Avoid asynchronous installation when batch-executing all notebook statements,
#       as execution will continue even without installation being complete.
#
# Can install by passing a list of packages/versions
#   Or 
# install using a requirements.txt file.

# For this demo, 
# this code block will collect the current user's package versions
# for installation into the container
# when using dataframe.apply(), it pandas and dill are required
# to reduce issues, match the version between client and container

# import these functions inside of a function namespace
def get_versions(pkgs):
    local_v_pkgs = []
    for p in pkgs:

        #fix up any hyphened package names
        p_fixed = p.replace('-', '_')

        #import the packages and append the strings to the list
        exec(f'''import {p_fixed}; local_v_pkgs.append('{p}==' + str({p_fixed}.__version__))''')
    return local_v_pkgs

v_pkgs = get_versions(pkgs)



# check to see if these packages need to be installed
# by comparing the len of the intersection of the list of required packages with the installed ones
if not len(set([x.split('==')[0] for x in pkgs]).intersection(demo_env.libs['name'].to_list())) == len(pkgs):
    
    # pass the list of packages - split off any extra info from the version property e.g., plus sign
    claim_id = demo_env.install_lib([x.split('+')[0] for x in v_pkgs], asynchronous=True)
else:
    print(f'All required packages are installed in the {oaf_name} environment')

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Monitor library installation status</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optionally - users can monitor the library installation status using the cell below:
</p> 

In [ ]:
# Check the status of installation using status() API.
# Create a loop here for demo purposes
try: 
    claim_id
    ipydisplay(demo_env.status(claim_id))
    stage = demo_env.status(claim_id)['Stage'].iloc[-1]
    while stage == 'Started':
        stage = demo_env.status(claim_id)['Stage'].iloc[-1]
        clear_output()
        ipydisplay(demo_env.status(claim_id))
        sleep(5)
except NameError:
    print('No installations to monitor')

    
# Verify the Python libraries have been installed correctly.
ipydisplay(demo_env.libs)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Download and install model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Open Analytics Framework containers do not have open access to the external network, which contributes to a very secure runtime environment.  As such, users will load pre-trained models using the below APIs.  For illustration purposes, the following code will check to see if the model archive exists locally and if it doesn't, will import and download it by creating a model object.  The archive will then be created and installed into the remote environment.
</p> 

In [ ]:
# check to see if the model needs to be downloaded/archived

# construct the file name for the model:
model_base = model_name.split('/')[1]
model_path = './' + model_base


if not os.path.isfile(f'{model_base}.zip'):

    from transformers import pipeline, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
    from transformers import AutoTokenizer, AutoConfig
    import torch

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = model = AutoModelForSequenceClassification.from_pretrained(model_name)
    sentiment_task = pipeline("sentiment-analysis", model = model , tokenizer = tokenizer, device = device)
    
    # Save model to the local directory - could also pick it up out of huggingface cache dir
    tokenizer.save_pretrained(model_path)
    model.save_pretrained(model_path)

    import shutil
    print('Creating Model Archive...')

    sentiment_task = pipeline("sentiment-analysis", model = model_name , tokenizer = model_name)
    shutil.make_archive(model_base, 
                        format = 'zip', 
                        root_dir = model_path)
else:
    print('Local model archive exists.')

# check to see if the model is already installed
try:
    if demo_env.models.empty: # no models installed at all
        print('Installing Model...')
        claim_id = demo_env.install_model(model_path = f'{model_base}.zip', asynchronous = True)
    elif not any(model_base in x for x in demo_env.models['Model']): #see if model is there
        print('Installing Model...')
        claim_id = demo_env.install_model(model_path = f'{model_base}.zip', asynchronous = True)
    else:
        print('Model already installed')
except Exception as e:
    if '''NoneType' object has no attribute 'empty''' in str(e):
        print('Installing Model...')
        claim_id = demo_env.install_model(model_path = f'{model_base}.zip', asynchronous = True)
        pass
    else:
        raise

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Monitor model installation status</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optionally - users can monitor the model installation status using the cell below:
</p> 

In [ ]:
# Check the status of installation using status() API.
# Create a loop here for demo purposes
try: 
    claim_id
    ipydisplay(demo_env.status(claim_id))
    stage = demo_env.status(claim_id)['Stage'].iloc[-1]
    while stage != 'File Installed':
        stage = demo_env.status(claim_id)['Stage'].iloc[-1]
        clear_output()
        ipydisplay(demo_env.status(claim_id))
        sleep(5)
except NameError:
    print('No installations to monitor')

    
# Verify the model has been installed correctly.
demo_env.refresh()
ipydisplay(demo_env.models)

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Conclusion - Environment Management</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The preceding demo showed how users can perform a <b>one-time</b> configuration task to prepare a custom environment for analytic processing at scale.  Once this configuration is complete, these containers can be re-used in ad-hoc development tasks, or used for operationalizing analytics in production.</p>

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Sentiment Analysis using the Analytic Cluster
  <br>
    </p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration will utilize the custom container environment built in the prior demonstration.  A user function to analyze customer sentiment will be passed to the Analytic GPU Cluster for execution at scale.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Compute Clusters are <b>elastic and ephemeral</b>; they offer automated scale up and down based on demand, and can be turned on or off on a schedule or programmatically.  This allows for greater cost and resource efficiency, where users can turn the GPU-enable cluster on for the duration of the transformation job, then shut it down when the processing is complete.</p>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Data Preparation Workflow</b></p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '30%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Write a <b>local</b> function that leverages a pre-trained Language Model to analyze sentiment</li>
                <br>
                <li>Push this processing to the <b>GPU Analytic Cluster</b> for processing at scale</li>
                <br>
                <li><b>Optimize</b> the analysis function to run at scale to maximize GPU throughput</li>
                <br>
                <li><b>Execute batch sentiment sxtraction</b> to store the results for further analytic processing</li>
            </ol>
        </td>
        <td style = 'vertical-align:top'>
            <img src = 'images/local_remote_functions.png'>
        <br>
        <img src = 'images/optimize.png'></td>
    </tr>
</table>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Check connection</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Check database and UES connection</p> 

In [ ]:
# check for existing connection
eng = check_and_connect(host=host, username=username, password=password, compute_group = compute_group)
print(eng)
    

# check to see if there is a valid UES auth
# if not, authenticate
try:
    demo_env = get_env(oaf_name)

except Exception as e:
    if '''NoneType' object has no attribute 'value''' or '''Failed to execute get_env''' in str(e): #UES auth expired/required
        if set_auth_token(ues_url = ues_url, username = username, pat_token = pat_token, pem_file = pem_file):
            print('UES Authentication successful')
            try:
                demo_env = get_env(oaf_name)
                pass
            except Exception as l:
                if f'''User environment '{oaf_name}' not found''' in str(l):
                    print('User environment not found')
                    pass
                else:
                    raise
        else:
            print('UES Authentication failed, check URL and account info')
        pass
    elif f'''User environment '{oaf_name}' not found''' in str(e):
        print('User environment not found')
        pass
    else:
        raise


# check cluster status
check_cluster_start(compute_group = compute_group)

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1.  Create a client-side embedding function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The goal of this exercise is to create a client-side function which can be "pushed" to the analytic cluster for processing at scale.  There are a few minor enhancements here to improve performance and usability in this remote environment:</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Imports</b>.  By default, the teradataml python client will package the code, objects, and dependencies and serialize it to the analytic cluster.  Users can make this more efficient by staging these objects before hand (done in the first demo of this notebook).  <b>Important</b> - place the larger libraries (sentence_transformers and torch) inside the function so they won't be registered as new dependencies that need to be installed.</li>
    <li><b>GPU Drivers</b>.  Since this function will run both on the client (CPU) and cluster (GPU), place a conditional to check</li>
    </ul>


In [ ]:
def get_sentiment(row):
    import warnings
    warnings.filterwarnings('ignore')
    from transformers import pipeline, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
    from transformers import AutoTokenizer, AutoConfig
    import torch
    from transformers.utils import logging
    logging.set_verbosity_error() 
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model_name = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
    if os.environ.get('OPENAF_MODELS_DIR'):
        model_path = os.environ.get('OPENAF_MODELS_DIR') + '/' + model_name.split('/')[1]
    else:
        model_path = './' + model_name.split('/')[1]
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = model = AutoModelForSequenceClassification.from_pretrained(model_path)
    sentiment_task = pipeline("sentiment-analysis", model = model , tokenizer = tokenizer, device = device)

    s = pd.Series(sentiment_task(row['text'])[0])

    # concat them together
    row = pd.concat([row[['id']], s], axis = 0)
    
    return row

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.2.  Test the function on local data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Python pandas provides an apply() method to execute the function across all rows in the DataFrame</p>

In [ ]:
df = pd.DataFrame({'id':[1,2], 'text':['very nice!', 'very bad!']})

df.apply(get_sentiment, axis = 1)



<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Step 2. Push this function to the analytic cluster</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Similar to pandas, the teradataml package provides an apply() method which is called in a similar manner, except it runs <b>in parallel on</b> the cluster, leveraging the GPU infrastructure and the MPP processing capabilities of Vantage.</p>

    

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.1 - Inspect the Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Simple DataFrame methods to show the data.  A teradataml DataFrame behaves like a normal pandas DataFrame, with one significant difference in that it is a reference to data on the analytic database.  This allows developers to perform familiar data mangement operations on extremely large data sets as if the data is local.</p>

In [ ]:
tdf_calls = DataFrame('"demo_ofs"."cust_calls"')

ipydisplay(tdf_calls.head(1))

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.2 - Prepare to execute the apply method</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>When using the teradataml apply method,there are some differences in how data is passed in and out of the multiple runtime containers on the distributed nodes.  This offers a great deal of processing power, but also requires some additional considerations when calling the method:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Optional Return Types</b>.  If the data being returned by the function has different columns/data types than the input DataFrame, the user passes these as dictionary value to the keyword argument</li>
    <li><b>Data Preparation</b>.  Of course, data preparation and cleansing can be performed inside the function. However, the unique architecture of the <b>Teradata Vantage</b> analytic database, is that users can execute powerful native data preparation functions that will operate at extreme scale and performance</li>
    </ol>

In [ ]:

types_dict = OrderedDict({})
types_dict['id'] = BIGINT()
types_dict['label'] = VARCHAR(20)
types_dict['score'] = FLOAT()

tdf = DataFrame.from_query('''SELECT TOP 10 id,
    CASE 
        WHEN text IS NULL THEN ' '
        ELSE regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(text , X'0d' , ' ') , X'0a' , ' ') , X'09', ' '), ',', ' '), '"', ' ')
    END text
    FROM demo_ofs.cust_calls;''')

tdf.head(2)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.3 - Execute the function on the nodes</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Call the apply method on the teradataml DataFrame.  This will push the function to the analytic nodes for processing in parallel.  Data is returned as another teradataml DataFrame, which represents the entire result set of the operation.  For this example, additional method arguments include:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Function</b>.  The name of the function to pass - this is the original client-side function written above</li>
    <li><b>Environment</b>.  The custom runtime environment with the models and dependencies loaded</li>
    <li><b>Return types</b>. OrderedDict that represents the column names and data types</li>
    </ol>

In [ ]:
%%time
# create a new dataframe representing the embeddings
tdf_sentiment = tdf.apply(lambda row: get_sentiment(row),
                    env_name = demo_env, 
                    returns = types_dict)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Check the data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now - users can create a table with these embeddings, use it in additional analytics, or operationalize this as part of a pipeline.</p>

In [ ]:
ipydisplay(tdf_sentiment.sample(2))

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Conclusion - executing custom code against GPU-enabled analytic clusters</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The preceding demo showed how users can use simple, familiar patters to execute powerful AI models <b>at scale</b> for development and operational processing.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Stop the Cluster</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Hibernate the environment if desired</p>

In [ ]:
check_cluster_stop(compute_group)

<hr>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Demo 3 - Operationalizing AI-powered analytics
  <br>
    </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration will illustrate how developers can take the next step in the process to <b>operationalize</b> this processing, enabling the entire organization to leverage AI across the data lifecycle, including</p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '30%'>
           <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
               <li><b>Prepare the environment</b>.  Package the scoring function into a more robust program, and stage it on the remote environment</li>
            <br>
            <br>
               <li><b>Python Pipeline</b>.  Execute the function using Python methods, and commit the resulting transformations to database tables.  Test the native ClearScape Analytics Functions</li>
            <br>
            <br>
               <li><b>Operationalize</b>.  Simplify the analytic pipeline to support ongoing operational transformations, on-demand analytics, and third-party applications</li>
        </ol>
        </td>
        <td width = '20%'></td>
        <td style = 'vertical-align:top'><img src = 'images/OAF_Ops.png' width = 350 ></td>
    </tr>
</table>


<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Check connection</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Reconnect to the database, UES, and start cluster if necessary<get_context()/p> 

In [ ]:
# check for existing connection
eng = check_and_connect(host=host, username=username, password=password, compute_group = compute_group)
print(eng)
    

# check to see if there is a valid UES auth
# if not, authenticate
try:
    demo_env = get_env(oaf_name)

except Exception as e:
    if '''NoneType' object has no attribute 'value''' in str(e): #UES auth expired/required
        if set_auth_token(ues_url = ues_url, username = username, pat_token = pat_token, pem_file = pem_file):
            print('UES Authentication successful')
            try:
                demo_env = get_env(oaf_name)
                pass
            except Exception as l:
                if f'''User environment '{oaf_name}' not found''' in str(l):
                    print('User environment not found')
                    pass
                else:
                    raise
        else:
            print('UES Authentication failed, check URL and account info')
        pass
    elif f'''User environment '{oaf_name}' not found''' in str(e):
        print('User environment not found')
        pass
    else:
        raise


# check cluster status
check_cluster_start(compute_group = compute_group)

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1  Create a server-side embedding function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The goal of this exercise is to create a <b>server-side</b> function which can be staged on the analytic cluster.  This offers many improvements over the method used above;</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Performance</b>.  Staging the code and dependencies in the container environment reduces the amount of I/O, since the function doesn't need to get serialized to the cluster when called</li>
    <li><b>Operationalization</b>.  The execution pipeline can be encapsulated into a SQL statement, which allows for seamless use in ETL pipelines, dashboards, or applications that need access</li>
    <li><b>Flexibility</b>. Developers can express much greater flexibility in how the code works to optimize for performance, stability, data cleanliness or flow logic</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>These benefits do come with some amount of additional work.  Developers need to account how data is passed in and out of the code runtime, and how to pass it back to the SQL engine to assemble and return the final resultset.  Code is executed when the user expresses an <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/SQL-Reference/SQL-Operators-and-User-Defined-Functions/Table-Operators/APPLY'>APPLY SQL function</a>;</p> 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Input Query</b>.  The APPLY function takes a SQL query as input.  This query can be as complex as needed and include data preparation, cleansing, and/or any other set-based logic necessary to create the desired input data set.  This complexity can also be abstracted into a database view.  When using the teradata client connectors for Python or R, thise query is represented as a DataFrame or tibble.</li>
    <li><b>Pre-processing</b>.  Based on the query plan, data is retrieved from storage (cache, block storage, or object storage) and the input query is executed.</li>
    <li><b>Distribution</b>.  Input data can be partitioned and/or ordered to be processed on a specific container or collection of them.  For example, the user may want to process all data for a single post code in one partition, and run thousands of these in parallel.  Data can also be distributed evenly across all units of parallelism in the system</li>
    <li><b>Input</b>.  The data for each container is passed to the runtime using tandard input (stdin)</li>
    <li><b>Processing</b>.  The user's code executes, parsing stdin for the input data</li>
    <li><b>Output</b>.  Data is sent out of the code block using standard output (stdout)</li>
    <li><b>Resultset</b>.  Resultset is assembled by the analytic database, and the SQL query returns</li>
    </ol>


<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Example server-side code block</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This is the python script used in the demonstration.  It is saved to the filesystem as "embedding.py".  Note here there have been multiple optimizations to improve performance at scale; specifically using batching as compared to dataframe.apply() which is much slower.</p> 


```python

#!/usr/bin/env python3
import sys, csv, os
import warnings
import torch 
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers.utils import logging
warnings.simplefilter('ignore')
logging.set_verbosity_error() 

# model_name = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
model_path = os.environ.get('OPENAF_MODELS_DIR') + '/twitter-roberta-base-sentiment-latest'

# 1. use the csv reader to parse comma-separated input
# 2. construct the Dataframe from the resulting dictionary

colNames = ['id', 'text']
d = csv.DictReader(sys.stdin.readlines(), fieldnames = colNames)
df = pd.DataFrame(d, columns = colNames)

# Use try...except to produce an error if something goes wrong in the try block
try:
    # Exit gracefully if DataFrame is empty
    # It is possible some partitions won't get any data
    if df.empty:
        sys.exit()
        
    
    #################################
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = model = AutoModelForSequenceClassification.from_pretrained(model_path)
    sentiment_task = pipeline("sentiment-analysis", model = model , tokenizer = tokenizer, device = device)
    #################################

    def get_sentiment(row):
        #sentiments = sentiment_task(texts, batch_size = 10)
        return pd.Series(sentiment_task(row['text'])[0])
        #return sentiments

   # texts = df['text'].tolist()
   # sentiments = get_sentiment(texts)
    
    # sentiment_df = pd.DataFrame(sentiments, columns=['result'])
    
    s_df = df.apply(get_sentiment, axis = 1)
    df = pd.concat([df[['id']], s_df], axis=1)
    # Egress results to the Database through standard output.
    df.to_csv(sys.stdout, index=False, header=False)
        
# raise any errors back to the SQL engine
except (SystemExit):
    # Skip exception if system exit requested in try block
    pass
except:    # Specify in standard error any other error encountered
    print("Script Failure :", sys.exc_info()[0], file=sys.stderr)
    raise
    sys.exit()

```

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 3.  Install the file and any additional artifacts</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the install_file() method to install this python file to the container.  As a reminder, this container is persistent, so these steps need only be done infrequently.</p> 

In [ ]:
demo_env.install_file('Sentiment_Extractor_twitter_roberta_base.py', replace = True)

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 4.  Call the APPLY function </b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function can be executed in two ways;</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b><a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Teradata-Package-for-Python-on-VantageCloud-Lake/Working-with-Open-Analytics/teradataml-Apply-Class-for-APPLY-Table-Operator'>Python</a></b> by calling the Apply() module function</li>
    <li><b><a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/SQL-Reference/SQL-Operators-and-User-Defined-Functions/Table-Operators/APPLY'>SQL</a></b> which allows for broad adoption across the enterprise</li>
    </ul>
    

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>APPLY using Python</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The process is as follows</p> 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Construct a dictionary that will define the return columns and data types</li>
    <li>Construct a teradataml DataFrame representing the data to be processed - note this is a "virtual" object representing data and logic <b>in-database</b></li>
    <li>Execute the module function.  This constructs the function call in the database, but does not execute anything.  Note the Apply function takes several arguments - the input data, environment name, and the command to run</li>
    <li>In order to execute the function, an "execute_script()" method must be called.  This method returns the server-side DataFrame representing the complete operation.  This DataFrame can be used in further processing, stored as a table, etc.</li>
    </ol>
    

In [ ]:
types_dict = OrderedDict({})
types_dict['id'] = BIGINT()
types_dict['label'] = VARCHAR(100)
types_dict['score'] = FLOAT()

view_query = '''REPLACE VIEW Cleaned_Text_V AS
                SELECT id,
                    CASE 
                        WHEN text IS NULL THEN ' '
                        ELSE regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(text , X'0d' , ' ') , X'0a' , ' ') , X'09', ' '), ',', ' '), '"', ' ')
                    END text
                FROM demo_ofs.cust_calls;'''
execute_sql(view_query)

tdf = DataFrame('Cleaned_Text_V')

In [ ]:
apply_obj = Apply(data = tdf,
                  apply_command = 'python Sentiment_Extractor_twitter_roberta_base.py',
                  returns = types_dict,
                  env_name = oaf_name,
                 )

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Execute the function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>call execute_script(), and return a single record to the client to check the data.</p> 

In [ ]:
%%time
# all records
ipydisplay(apply_obj.execute_script().sample(1))

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 5. Persist the resultset</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Simple methods for saving the results to a table for reuse.</p> 

In [ ]:
%%time
copy_to_sql(Apply(data = tdf,
                  apply_command = 'python Sentiment_Extractor_twitter_roberta_base.py',
                  returns = types_dict,
                  env_name = oaf_name
                 ).execute_script(), table_name = 'customer_sentiment', temporary = False, if_exists = 'replace')
execute_sql('SELECT COUNT(*) FROM customer_sentiment;').fetchall()

In [ ]:
tdf_sentiment = DataFrame('customer_sentiment')
tdf_sentiment

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>APPLY using SQL</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The process is much the same, except the benefit is that this SQL can be used by a wide range of tools, applications, and dashboards; as well as automated processes.  Construct the statement using the following values:</p> 
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>
        <b>ON</b> Clause<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li>Input table or query</li>
        <li>Hash or Partition column(s)</li>
        <li>Order by and/pr local order by directives</li>
        <li>Return columns and data types</li></ul></li>
    <li>Any partition column(s)</li>
    <li>The shell command to run</li>
    <li>Additional functional arguments</li>
    </ul>

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Simplify the SQL using Views</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a view that represents the data preparation steps.  In this example, the preparation tasks include removing NULLs and any special characters.  Note the same SQL used to contstruct the teradataml DataFrame is used here.</p> 

In [ ]:
qry = '''
    REPLACE VIEW prepared_data_V AS
    
SELECT id,
    CASE 
        WHEN text IS NULL THEN ' '
        ELSE regexp_replace(regexp_replace(regexp_replace(regexp_replace(regexp_replace(text , X'0d' , ' ') , X'0a' , ' ') , X'09', ' '), ',', ' '), '"', ' ')
    END text
    FROM demo_ofs.cust_calls;
    
    '''

# execute the sql against the database
execute_sql(qry)

# check the data
execute_sql('SELECT TOP 1 * FROM prepared_data_V;').fetchall()

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Construct the APPLY Query</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the view as input data, provide "returns" payload and required parameters.</p> 

In [ ]:
qry = f'''
SELECT * FROM Apply(
    ON prepared_data_V
    PARTITION BY ANY
    
    returns(id BIGINT, label VARCHAR(100), score FLOAT) 
    USING
    
    APPLY_COMMAND('python Sentiment_Extractor_twitter_roberta_base.py')
    ENVIRONMENT('{oaf_name}')
    STYLE('csv')
    delimiter(',') 
) as d;'''


# execute the query on the server, and return the first record
execute_sql(qry).fetchone()

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Simplify the APPLY using a view</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Construct a database view using the query above.  Now this simple query can be embedded in operational processing, ETL functions, or the like.</p> 

In [ ]:
view_qry = f'''REPLACE VIEW simplified_sentiment_V AS {qry}
'''

execute_sql(view_qry);

In [ ]:
df = pd.read_sql('SELECT TOP 10 * FROM simplified_sentiment_V', eng)
df

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Conclusion - Operationalizing AI-powered analytics</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The preceding demo showed two methods for operationalizing the model execution; using python syntax, or embedding it as a simplified SQL view.  The former allows for developers and data scientists to easily embed this processing in their existing or new applications and workflows.  The latter allows for broad, democratized adoption across the data lifecycle and enterprise - enabling this analytic processing in ETL for data prep and transformation tasks, and in production to power dashboards and/or BI tools.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Cleanup</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Stop the cluster, remove the environment (if desired), and drop views created in the demonstrations.  Finally, disconnect from the database.</p>

In [ ]:
res = check_cluster_stop(compute_group)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Optional - remove environment and artifacts</b></p>

In [ ]:
# uninstall the libraries from the environment first before removing it
demo_env.uninstall_lib(libs = demo_env.libs['name'].to_list())
remove_env(oaf_name)

In [ ]:
execute_sql('DROP VIEW simplified_sentiment_V;');
execute_sql('DROP VIEW prepared_data_V;');
execute_sql('DROP VIEW Cleaned_Text_V;');
execute_sql('DROP TABLE customer_sentiment;');

In [ ]:
remove_context()